###Atharva Mankame [21BAI1780]

### Implementing a convolutional neural network with k-fold cross validation (MNIST dataset)

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.datasets import mnist
from keras.utils import to_categorical

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
X_train = X_train[0:10000,:]
y_train = y_train[0:10000]
X_test = X_test[0:2000,:]
y_test = y_test[0:2000]

In [4]:
print(X_train.shape)
print(X_test.shape)

(10000, 28, 28)
(2000, 28, 28)


In [5]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255   # 60000,28,28,1

In [6]:
 # one hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [9]:
cvscores = []
for train, test in kfold.split(X_train, y_train.argmax(axis=1)):
    model = create_cnn_model()
    model.fit(X_train[train], y_train[train], epochs=10, batch_size=32, verbose=1)
    scores = model.evaluate(X_train[test], y_train[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

Epoch 1/10
250/250 [==============================] - 17s 51ms/step - loss: 0.4366 - accuracy: 0.8770
Epoch 2/10
250/250 [==============================] - 7s 29ms/step - loss: 0.1561 - accuracy: 0.9545
Epoch 3/10
250/250 [==============================] - 6s 24ms/step - loss: 0.0940 - accuracy: 0.9736
Epoch 4/10
250/250 [==============================] - 5s 21ms/step - loss: 0.0574 - accuracy: 0.9840
Epoch 5/10
250/250 [==============================] - 5s 20ms/step - loss: 0.0411 - accuracy: 0.9884
Epoch 6/10
250/250 [==============================] - 6s 25ms/step - loss: 0.0247 - accuracy: 0.9937
Epoch 7/10
250/250 [==============================] - 5s 21ms/step - loss: 0.0155 - accuracy: 0.9962
Epoch 8/10
250/250 [==============================] - 6s 25ms/step - loss: 0.0094 - accuracy: 0.9981
Epoch 9/10
250/250 [==============================] - 5s 19ms/step - loss: 0.0043 - accuracy: 0.9998
Epoch 10/10
250/250 [==============================] - 6s 23ms/step - loss: 0.0032 - accur

In [10]:
print(f"Average Accuracy: {np.mean(cvscores):.2f}% (+/- {np.std(cvscores):.2f}%)")

Average Accuracy: 97.16% (+/- 0.09%)
